# MTH3302 : Méthodes probabilistes et statistiques pour l'I.A.

Jonathan Jalbert<br/>
Professeur agrégé au Département de mathématiques et de génie industriel<br/>
Polytechnique Montréal<br/>


# Chapitre 2 - Introduction à la régression linéaire

Ce calepin Jupyter accompage le chapitre 2 des notes de cours.

Chargeons d'abord les librairies nécessaires.

In [1]:
using CSV, DataFrames             # Pour charger et organiser les données
using Gadfly                      # Pour générer des graphiques
using Distributions               # Pour utiliser les distributions statistiques
using LinearAlgebra               # Pour utiliser les fonctions d'algèbre linéaire
using Combinatorics               # Pour énumérer toutes les possibilités

LoadError: ArgumentError: Package Combinatorics not found in current path.
- Run `import Pkg; Pkg.add("Combinatorics")` to install the Combinatorics package.

Si les librairies ne sont pas installées, excécutez le code suivant :

```julia
using Pkg
Pkg.add(["CSV", "DataFrames", "Combinatorics", "Gadfly", "Distributions"])
```

# Chargement des données

In [ ]:
# Chargement du fichier de données 

filename = joinpath(dirname(@__FILE__), "Data", "FF_emissions.csv")

data = CSV.read(filename, DataFrame)

# Les colonnes sont renommées avec des noms plus courts
rename!(data, :FF_emissions => :FF)
rename!(data, :LUC_emissions => :LUC)
rename!(data, :ocean_sink => :Ocean)
rename!(data, :land_sink => :Land)
rename!(data, :T_anomaly => :T)

# Les océans et les strates végétales absorbent le carbone
data[!,:Ocean] = -data[:,:Ocean]
data[!,:Land] = -data[:,:Land]

# Affichage des 5 premières lignes du tableau de données
first(data,5)

In [ ]:
# Extraction des données du DataFrame
# Utile pour écrire simplement les calculs matriciels

x₁ = data[:,:FF]     # Vecteur de la variable explicative 1
x₂ = data[:,:LUC]    # Vecteur de la variable explicative 2
x₃ = data[:,:Ocean]  # Vecteur de la variable explicative 3
x₄ = data[:,:Land]   # Vecteur de la variable explicative 4
y = data[:,:T]       # Variable d'intérêt
n = length(y)        # Nombre d'observations

# Analyse exploratoire

Étape permettant d'obtenir une vision globale du jeu de données. Elle peut également permettre de découvrir des dépendances entre les variables. L'analyse exploratoire peut s'affectuer à l'aide de graphiques et de statistiques simples.

In [ ]:
# Traçage de l'anomalie de température en fonction des années

Gadfly.set_default_plot_size(12cm, 8cm)
fig1 = plot(data, x=:Year, y=:T, Geom.line,
    Coord.Cartesian(xmin=1959, xmax=2015), Guide.xticks(ticks=1960:10:2015),
    Guide.xlabel("Année"),
    Guide.ylabel("Anomalie de température (°C)")) 

In [ ]:
# Traçage des composantes du cycle du carbone en fonction des années

Gadfly.set_default_plot_size(14cm, 8cm)
fig2 = plot(stack(data, [:FF, :LUC, :Ocean, :Land]), x=:Year, y=:value, color=:variable, Geom.line,
    Coord.Cartesian(xmin=1959, xmax=2015), Guide.xticks(ticks=1960:10:2015),
    Guide.xlabel("Année"),
    Guide.ylabel("Carbone émis (Gt)"))

# 2.1 Modèle de régression linéaire simple

Pour commencer, on utilise seulement les émissions par combustion de combustibles fossibles 
comme variable explicative.

### Estimation des paramètres du modèle de régression linéaire simple

In [ ]:
# Calcul des statistiques utiles
x̄ = mean(x₁)
ȳ = mean(y)

# Estimation des paramètres
β̂₁ = sum( (x₁[i] - x̄)*(y[i] - ȳ) for i=1:n) / sum( (x₁[i] - x̄)^2 for i=1:n )

β̂₀ = ȳ - β̂₁*x̄

println("L'ordonnée à l'origine estimée est β̂₀=", β̂₀)
println("La pente de la droite de régression estimée est β̂₁=", β̂₁)


### Affichage de la droite de régression

In [ ]:
Gadfly.set_default_plot_size(12cm, 8cm)
plot(data, x=:FF, y=:T, Geom.point, 
    intercept = [β̂₀], slope = [β̂₁], Geom.abline(color="red", style=:dash),
    Coord.cartesian(xmin=2, xmax=10),
    Guide.xlabel("Carbone émis par la combustion de FF (Gt)"),
    Guide.ylabel("Anomalie de température (°C)"))

# 2.2 Modèle de régression linéaire multiple

Dans cette section, toutes les variables explicatives sont utilisées pour expliquer les anomalies de température.

### 2.2.2 Estimation des paramètres du modèle de régression multiple

Estimation des paramètres à l'aide de la méthode des moindres carrés.

In [ ]:
# Construction de la matrice de structure (Notez la colonnes de 1)
X = hcat(ones(n),x₁,x₂,x₃,x₄)

# Estimation des paramètres du modèle de régression
β̂ = (X'X)\X'y

println("Les estimations des coefficient de régression sont β̂ = ", β̂)

### 2.2.3 Prévision de l'anomalie de température

Supposons que l'on veut savoir qu'elle sera l'anomalie de température si on émet les quantités suivantes :

 - 10 Gt de C par FF
 - 1.5 Gt de C par LUC
 - -1 Gt de C par Ocean
 - -1 Gt de C par Land

In [ ]:
# Définition du vecteur des variables explicatives pour lesquelles on veut estimer la variable d'intérêt
x₀ = [1 10 1.5 -1 -1]

# Estimation de l'anomalie de température
Ŷ₀ = x₀*β̂

println("L'anomalie de température estimée correspondante est de ",Ŷ₀)

# 2.3 Indice de qualité du modèle de régression

## 2.3.1 Décomposition de la variabilité

In [ ]:
ȳ = mean(y)
ŷ = X*β̂

e = y-ŷ

SST = sum( (y .- ȳ).^2 )
SSR = sum( (ŷ .- ȳ).^2 )
SSE = sum(e.^2)

println("La variabilité totale est de ", SST)
println("La variabilité expliquée est de ", SSR)
println("La variabilité résiduelle est de ", SSE)

On peut vérifier que $e^\top ŷ = 0$ et $e^\top 1 = 0$.

In [ ]:
println("e'ŷ = ",e'*ŷ) 
println("e'1 = ",e'*ones(n)) 

## 2.3.2 Coefficient de détermination

In [ ]:
R² = SSR/SST
println("Le coefficient de détermination est de ", R²)

# 2.5 Propriétés des estimateurs

## 2.5.2 Estimation de la variance de l'erreur

In [ ]:
# Nb de variables explicatives
p = size(X,2)-1 # la colonne de 1 ne constitue pas une variable explicative

# Estimation de la variance de l'erreur
σ̂² = 1/(n-p-1)*sum( e.^2 )

println("σ̂² = $σ̂²")

## 2.5.4 Estimation de la matrice de covariance des estimateurs 

In [ ]:
V = σ̂² * inv(X'X)

# 2.6 Tests d'hypothèses et intervalles de confiance

## 2.6.1 Test sur l'importance de la régression

In [ ]:
# Calcul de la statistique observée du test
F₀ = SSR/SSE * (n-p-1)/p

xx = 0:.1:100
pd = FDist(p, n-p-1) 
f = pdf.(pd,xx)

density= layer(x=xx, y=f, Geom.line, Theme(default_color="deepskyblue"))
statObs = layer(x=[F₀], y=[0], Geom.point, Theme(default_color="red") )

Gadfly.set_default_plot_size(21cm, 8cm)
plot(density, statObs,
    Guide.manual_color_key("Légende", ["Densité sous H₀", "Stat obs"], ["deepskyblue","red"]),
    Coord.Cartesian(xmin=0,xmax=85), Guide.xlabel("x"), Guide.ylabel("f"))

In [ ]:
# Première façon de tester l'importance de la régression : comparaison avec le seuil critique

α = 0.05
point_critique = quantile(pd,1-α)
if F₀ > point_critique
    println("La régression est significative au seuil de $α car F₀ > point critique.
        On a en effet que F₀ = $F₀ > $point_critique)")
else
    println("La régression n'est pas significative au seuil de $α car F₀ < point critique.
        On a en effet que F₀ = $F₀ < $point_critique)")
end

In [ ]:
# Deuxième façon de tester l'importance de la régression : calcul de la valeur-p

α = 0.05
# calcul de la probabilité d'observer une statistique plus extrême que celle obtenue
seuil_observe = ccdf(pd,F₀) # la fonction ccdf correspond à 1-cdf()
if seuil_observe < α
    println("La régression est significative au seuil de $α car le seuil observé du test est plus petit que α.
        On a en effet que seuil observé = $seuil_observe < $α")
else
    println("La régression n'est pas significative au seuil de $α car le seuil observé du test est plus grand que α.
        On a en effet que seuil observé = $seuil_observe > $α")
end

## 2.6.2 Intervalles de confiance sur les coefficients de régression

In [ ]:
# calcul du quantile de la loi de Student
tₐ = quantile(TDist(n-p-1),.975)

# Affichage des intervalles de confiance
coeff_names = ["β₀", "β₁", "β₂", "β₃", "β₄"]
for j=0:p
    se = tₐ*sqrt(V[j+1,j+1])
   println("L'intervalle de confiance de niveau 95% pour $(coeff_names[j+1]) est [$(β̂[j+1] - se) , $(β̂[j+1] + se)]")
end

## 2.6.3 Intervalle de confiance sur une prédiction

Reprenons la prévision de la section 2.2.3. Calculons maintenons un intervalle de confiance de niveau 95 % pour cette prévision.

In [ ]:
# Calcul de la marge d'erreur
se = quantile(TDist(n-p-1),.975)*sqrt( σ̂²*(1 .+x₀/(X'X)*x₀' )

println("L'intervalle de confiance de Ŷ₀ est [$(Ŷ₀[1] - se[1]) , $(Ŷ₀[1] + se[1])]")

# 2.7 Validation des hypothèses de la régression linéaire

Analyse visuelle des résidus

In [ ]:
function henryplot(y::Vector{<:Real})

    n = length(y)
    ysorted = sort(y)

    p = ( collect(1:n) .- .5 ) /n

    fd = fit(Normal,y)

    q = quantile.(fd,p)

    plot(x=ysorted, y=q, Geom.point,
    Guide.xlabel("Empirical quantiles"), Guide.ylabel("Estimated quantiles"),
    Theme(discrete_highlight_color=c->nothing),
    Geom.abline(color="red"))


end

In [ ]:
# Stockage des variables dans un DataFrame pour un affichage plus facile
df = DataFrame(Ŷ = ŷ, e = e)

Gadfly.set_default_plot_size(21cm, 8cm)
f1 = plot(df, x = :Ŷ, y = :e, Geom.point, Coord.Cartesian(xmin=-.1,xmax=1,ymin=-.3,ymax=.3))
f2 = henryplot(e)

hstack([f1, f2])


In [ ]:
# Exemple de droite de Henry lorsque la distribution n'est pas normale


Gadfly.set_default_plot_size(12cm, 8cm)
henryplot(rand(Gamma(1,1),30))

# 2.9 Multicolinéarité

Vérification de la multicolinéarité avec le VIF.

In [ ]:
variables = names(data)[2:5]

df = DataFrame(Variable = String[], VIF = Float64[])

for variable in combinations(variables,1)
    
#     println(variable[])
   
    y = data[:, variable[] ]
    ȳ = mean(y)
    
    X = hcat(ones(n), Matrix(data[:, setdiff(variables, variable)]) )
    
    β̂ = X\y
    ŷ = X*β̂

    e = y-ŷ

    SSE = e'*e
    SST = sum( (y .- ȳ).^2 )
    
    R² = 1 - SSE/SST
    
    VIF = 1/(1-R²)
    
    push!(df, [variable[], VIF])
    
end

df


# 2.10 Sélection des variables explicatives

Dans cette section, on parcourt tous les modèles possibles. Il y en a 16 (en fait 15 si on exclut la possibilité où aucune des variables explicatives n'est sélectionnée). On calcule le coefficient de détermination ajusté de chacun de ces modèles pour déterminer le meilleur.

On constatera que le meilleur modèle est celui utilisant les 4 variables explicatives.

In [ ]:

y = data[:, :T]
ȳ = mean(y)
SST = sum( (y .- ȳ).^2 )


variables = names(data)[2:5]

df = DataFrame(Variable = Vector{String}[], R² = Float64[])

for variable in combinations(variables)
    
    p = length(variable)
    
    X = hcat(ones(n), Matrix(data[:, variable]))
    
    
    β̂ = X\y
    ŷ = X*β̂

    e = y-ŷ

    SSE = e'*e

    R²aj =  1 - SSE/SST * (n-1)/(n-p-1)
    
    push!(df, [variable, R²aj])
#     println(variable)
    
end

sort(df, :R², rev=true)

## On peut reproduire les résultats avec la librairie GLM

In [ ]:
using Pkg
Pkg.add("GLM")

In [ ]:
using GLM

lm(@formula(T ~ FF + LUC + Ocean + Land), data)

In [ ]:
using GLM

lm(@formula(T ~ FF), data)

In [ ]:
# Construction de la matrice de structure (Notez la colonnes de 1)
X = hcat(ones(n),x₁)

# Estimation des paramètres du modèle de régression
β̂ = (X'X)\X'y

In [ ]:
e = y - X*β̂

SSE = sum(e.^2)

# Nb de variables explicatives
p = size(X,2)-1 # la colonne de 1 ne constitue pas une variable explicative

# Estimation de la variance de l'erreur
σ̂² = 1/(n-p-1)*sum( e.^2 )

V = inv(X'X)

sqrt(σ̂²*V[2,2])

In [ ]:
t = β̂[2]/sqrt(σ̂²*V[2,2])

In [ ]:
T = TDist(n-p-1)

In [ ]:
2*ccdf(T, t)

In [ ]:
# zone critique
quantile(T, .975)

In [ ]:

density = layer(x -> pdf(T, x), -20, 20)
statObs = layer(x=[t], y=[0], Geom.point, Theme(default_color="red") )

Gadfly.set_default_plot_size(12cm, 8cm)
plot(density, statObs,
    Guide.manual_color_key("Légende", ["Densité sous H₀", "Stat obs"], ["deepskyblue","red"]),
    Coord.Cartesian(xmin=-20,xmax=20), Guide.xlabel("x"), Guide.ylabel("f"))


In [ ]:
β̂[2] + quantile(T, 0.975)*sqrt(σ̂²*V[2,2])

In [ ]:
temperatures = [280, 284, 292, 295, 298, 305, 308, 315]
viscosity = [770, 800, 840, 810, 735, 640, 590, 560]

In [ ]:
df = DataFrame(Temperature = temperatures, Viscosity = viscosity)

In [ ]:
plot(df, x=:Temperature, y=:Viscosity, Geom.point)

In [ ]:
n = nrow(df)

In [ ]:
X = hcat(ones(n), df.Temperature)

In [ ]:
X = hcat(ones(n), df.Temperature, df.Temperature.^2)

In [ ]:
function polynomial_structure_matrix(x::Vector{<:Real}, order::Int)
   
    n = length(x)
    
    X = ones(n)
    
    for i in 1:order
       X = hcat(X, x.^i)
    end
    
    return X
    
end

In [ ]:
X = polynomial_structure_matrix(df.Temperature, 7)

In [ ]:
β̂ = X\df.Viscosity

In [ ]:
ŷ = X*β̂

In [ ]:
function polynome(β̂::Vector{<:Real}, x::Real)
   
    ŷ = 0.0
    
    for i in eachindex(β̂)
        
        ŷ += β̂[i]*x.^(i-1)
        
    end
    
    return ŷ
    
end

In [ ]:
polynome(β̂, 280)

In [ ]:
plot(x-> polynome(β̂, x), 280, 315)